<a href="https://colab.research.google.com/github/rosehunnie/NLP/blob/main/RFMAnalysisAndCustomerSegmentCLTV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# RFM - per customer
# use that to put into a data mining model and predict revenue - (CLTV)
# Install required packages
!pip install pymc-marketing
!pip install lifetimes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.0 MB/s eta 0:00:00


In [2]:
# Customer Lifetime Value (CLV or CLTV)
# Buy till you die... (BG/NBD)
# data:  https://github.com/fenago/datasets/raw/main/Online%20Retail.xlsx

import lifetimes
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from lifetimes import BetaGeoFitter # BG/NBD
from lifetimes import GammaGammaFitter # Gamma-Gamma Model
from lifetimes.plotting import plot_frequency_recency_matrix
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [18]:
# Load the dataset
import pandas as pd
df = pd.read_csv("/content/sample_data/Electronic_sales_Sep2023-Sep2024.csv")

# Display the first few rows
print(df.head(20))


    Customer ID  Age  Gender Loyalty Member Product Type      SKU  Rating Order Status Payment Method  Total Price  Unit Price  Quantity Purchase Date Shipping Type                                 Add-ons Purchased  Add-on Total
0          1000   53    Male             No   Smartphone  SKU1004       2    Cancelled    Credit Card      5538.33      791.19         7    2024-03-20      Standard                     Accessory,Accessory,Accessory         40.21
1          1000   53    Male             No       Tablet  SKU1002       3    Completed         Paypal       741.09      247.03         3    2024-04-20     Overnight                                      Impulse Item         26.09
2          1002   41    Male             No       Laptop  SKU1005       3    Completed    Credit Card      1855.84      463.96         4    2023-10-17       Express                                               NaN          0.00
3          1002   41    Male            Yes   Smartphone  SKU1004       2    Complet

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Customer ID        20000 non-null  int64  
 1   Age                20000 non-null  int64  
 2   Gender             19999 non-null  object 
 3   Loyalty Member     20000 non-null  object 
 4   Product Type       20000 non-null  object 
 5   SKU                20000 non-null  object 
 6   Rating             20000 non-null  int64  
 7   Order Status       20000 non-null  object 
 8   Payment Method     20000 non-null  object 
 9   Total Price        20000 non-null  float64
 10  Unit Price         20000 non-null  float64
 11  Quantity           20000 non-null  int64  
 12  Purchase Date      20000 non-null  object 
 13  Shipping Type      20000 non-null  object 
 14  Add-ons Purchased  15132 non-null  object 
 15  Add-on Total       20000 non-null  float64
dtypes: float64(3), int64(4

In [20]:
df.describe()

,Customer ID,Age,Rating,Total Price,Unit Price,Quantity,Add-on Total
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10483.526550,48.994100,3.093950,3180.133419,578.631867,5.485550,62.244848
std,5631.732525,18.038745,1.223764,2544.978675,312.274076,2.870854,58.058431
min,1000.000000,18.000000,1.000000,20.750000,20.750000,1.000000,0.000000
25%,5478.000000,33.000000,2.000000,1139.680000,361.180000,3.000000,7.615000
50%,10499.500000,49.000000,3.000000,2534.490000,463.960000,5.000000,51.700000
75%,15504.000000,65.000000,4.000000,4639.600000,791.190000,8.000000,93.842500
max,19998.000000,80.000000,5.000000,11396.800000,1139.680000,10.000000,292.770000


In [21]:
# Ack extreme outliers for Quanity and UnitPrice
# Let's clean the data (however YOU decide to clean... just make sure it is logical)
df = df[df['Quantity'] > 0 ] # exclude the orders with 0 value
df = df[df['Unit Price'] > 0] # exclude the Unit Price with 0 value
df = df[~df['SKU'].str.contains("C",na=False)]  # C indicates the returned orders we don't want them as well

In [22]:
df.isnull().sum()

,0
Customer ID,0
Age,0
Gender,1
Loyalty Member,0
Product Type,0
SKU,0
Rating,0
Order Status,0
Payment Method,0
Total Price,0


In [23]:
df.dropna(inplace=True)

In [24]:
df.isnull().sum()

,0
Customer ID,0
Age,0
Gender,0
Loyalty Member,0
Product Type,0
SKU,0
Rating,0
Order Status,0
Payment Method,0
Total Price,0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15132 entries, 0 to 19998
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Customer ID        15132 non-null  int64  
 1   Age                15132 non-null  int64  
 2   Gender             15132 non-null  object 
 3   Loyalty Member     15132 non-null  object 
 4   Product Type       15132 non-null  object 
 5   SKU                15132 non-null  object 
 6   Rating             15132 non-null  int64  
 7   Order Status       15132 non-null  object 
 8   Payment Method     15132 non-null  object 
 9   Total Price        15132 non-null  float64
 10  Unit Price         15132 non-null  float64
 11  Quantity           15132 non-null  int64  
 12  Purchase Date      15132 non-null  object 
 13  Shipping Type      15132 non-null  object 
 14  Add-ons Purchased  15132 non-null  object 
 15  Add-on Total       15132 non-null  float64
dtypes: float64(3), int64(4), ob

In [26]:
def find_boundaries(df, variable,q1=0.05,q2=0.95):
    # the boundaries are the quantiles
    lower_boundary = df[variable].quantile(q1) # lower quantile
    upper_boundary = df[variable].quantile(q2) # upper quantile
    return upper_boundary, lower_boundary
def capping_outliers(df,variable):
    upper_boundary,lower_boundary =  find_boundaries(df,variable)
    df[variable] = np.where(df[variable] > upper_boundary, upper_boundary,
                       np.where(df[variable] < lower_boundary, lower_boundary, df[variable]))

In [28]:
capping_outliers(df,'Unit Price')
capping_outliers(df,'Quantity')

In [ ]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,397884.000000,397884,397884.000000,397884.000000
mean,8.868022,2011-07-10 23:41:23.511023360,2.675785,15294.423453
min,1.000000,2010-12-01 08:26:00,0.420000,12346.000000
25%,2.000000,2011-04-07 11:12:00,1.250000,13969.000000
50%,6.000000,2011-07-31 14:39:00,1.950000,15159.000000
75%,12.000000,2011-10-20 14:33:00,3.750000,16795.000000
max,36.000000,2011-12-09 12:50:00,8.500000,18287.000000
std,9.523425,NaN,2.275053,1713.141560


In [29]:
df.head()

,Customer ID,Age,Gender,Loyalty Member,Product Type,SKU,Rating,Order Status,Payment Method,Total Price,Unit Price,Quantity,Purchase Date,Shipping Type,Add-ons Purchased,Add-on Total
0,1000,53,Male,No,Smartphone,SKU1004,2,Cancelled,Credit Card,5538.33,791.19,7.0,2024-03-20,Standard,"Accessory,Accessory,Accessory",40.21
1,1000,53,Male,No,Tablet,SKU1002,3,Completed,Paypal,741.09,247.03,3.0,2024-04-20,Overnight,Impulse Item,26.09
3,1002,41,Male,Yes,Smartphone,SKU1004,2,Completed,Cash,3164.76,791.19,4.0,2024-08-09,Overnight,"Impulse Item,Impulse Item",60.16
4,1003,75,Male,Yes,Smartphone,SKU1001,5,Completed,Cash,41.50,20.75,2.0,2024-05-21,Express,Accessory,35.56
5,1004,41,Female,No,Smartphone,SKU1001,5,Completed,Credit Card,83.00,20.75,4.0,2024-05-26,Standard,"Impulse Item,Accessory",65.78


In [30]:
df['Total Price'] = df['Unit Price'] * df['Quantity']
df.head()

,Customer ID,Age,Gender,Loyalty Member,Product Type,SKU,Rating,Order Status,Payment Method,Total Price,Unit Price,Quantity,Purchase Date,Shipping Type,Add-ons Purchased,Add-on Total
0,1000,53,Male,No,Smartphone,SKU1004,2,Cancelled,Credit Card,5538.33,791.19,7.0,2024-03-20,Standard,"Accessory,Accessory,Accessory",40.21
1,1000,53,Male,No,Tablet,SKU1002,3,Completed,Paypal,741.09,247.03,3.0,2024-04-20,Overnight,Impulse Item,26.09
3,1002,41,Male,Yes,Smartphone,SKU1004,2,Completed,Cash,3164.76,791.19,4.0,2024-08-09,Overnight,"Impulse Item,Impulse Item",60.16
4,1003,75,Male,Yes,Smartphone,SKU1001,5,Completed,Cash,41.50,20.75,2.0,2024-05-21,Express,Accessory,35.56
5,1004,41,Female,No,Smartphone,SKU1001,5,Completed,Credit Card,83.00,20.75,4.0,2024-05-26,Standard,"Impulse Item,Accessory",65.78


In [31]:
df.columns

Index(['Customer ID', 'Age', 'Gender', 'Loyalty Member', 'Product Type', 'SKU', 'Rating', 'Order Status', 'Payment Method', 'Total Price', 'Unit Price', 'Quantity', 'Purchase Date', 'Shipping Type', 'Add-ons Purchased', 'Add-on Total'], dtype='object')

In [32]:
clv = lifetimes.utils.summary_data_from_transaction_data(df,'Customer ID','Purchase Date','Total Price',observation_period_end='2024-12-09')

In [33]:
clv.head()

,frequency,recency,T,monetary_value
Customer ID,,,,
1000,1.0,31.0,264.0,741.09
1002,0.0,0.0,122.0,0.00
1003,0.0,0.0,202.0,0.00
1004,0.0,0.0,197.0,0.00
1005,0.0,0.0,168.0,0.00


In [34]:
clv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10268 entries, 1000 to 19997
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   frequency       10268 non-null  float64
 1   recency         10268 non-null  float64
 2   T               10268 non-null  float64
 3   monetary_value  10268 non-null  float64
dtypes: float64(4)
memory usage: 401.1 KB


In [35]:
# let's filter so that we only catch customers who have made more than 1 purchase
clv = clv[clv['frequency']>1]

In [36]:
clv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 931 entries, 1038 to 19996
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   frequency       931 non-null    float64
 1   recency         931 non-null    float64
 2   T               931 non-null    float64
 3   monetary_value  931 non-null    float64
dtypes: float64(4)
memory usage: 36.4 KB


In [37]:
clv.head()

,frequency,recency,T,monetary_value
Customer ID,,,,
1038,2.0,77.0,154.0,2841.790000
1060,2.0,123.0,438.0,1532.280000
1079,2.0,327.0,412.0,2769.165000
1101,3.0,288.0,427.0,298.313333
1134,2.0,192.0,374.0,2319.800000


# Based off of this - how many purchases are expected by this loyal customer group: BGF

In [38]:
bgf = BetaGeoFitter(penalizer_coef=0.001)  # This will allow you to predict FUTURE purchase by taking into account F, R, T  penalizer is used to avoid overfitting
bgf.fit(clv['frequency'], clv['recency'], clv['T'])

<lifetimes.BetaGeoFitter: fitted with 931 subjects, a: 2.00, alpha: 555.60, b: 4.97, r: 6.04>

In [39]:
t = 180 # 6 month/180 day period
clv['expected_purc_6_months'] = bgf.conditional_expected_number_of_purchases_up_to_time(t, clv['frequency'], clv['recency'], clv['T'])
clv.sort_values(by='expected_purc_6_months',ascending=False).head(5)

,frequency,recency,T,monetary_value,expected_purc_6_months
Customer ID,,,,,
18304,6.0,192.0,276.0,3115.376667,1.243333
14497,4.0,204.0,285.0,4004.700000,1.045023
11476,4.0,260.0,339.0,7484.482500,1.028444
19378,4.0,245.0,334.0,2618.402500,0.980378
15679,4.0,219.0,310.0,4000.627500,0.978828


# Gamma-Gamma : This will tell me on average - how much each customer will spend.

In [40]:
clv[['frequency','monetary_value']].corr()

,frequency,monetary_value
frequency,1.000000,0.006442
monetary_value,0.006442,1.000000


In [41]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(clv["frequency"],
        clv["monetary_value"])

<lifetimes.GammaGammaFitter: fitted with 931 subjects, p: 3.09, q: 0.18, v: 2.99>

In [42]:
clv['six_Months_CLV']=ggf.customer_lifetime_value(bgf,
                                   clv["frequency"],
                                   clv["recency"],
                                   clv["T"],
                                   clv["monetary_value"],
                                   time=6,
                                   freq='D',
                                   discount_rate=0.01)
clv.sort_values('six_Months_CLV',ascending=False).head()

,frequency,recency,T,monetary_value,expected_purc_6_months,six_Months_CLV
Customer ID,,,,,,
15399,2.0,212.0,302.0,11396.800000,0.767291,9750.642036
13792,2.0,192.0,296.0,11525.280000,0.717379,9219.236277
11476,4.0,260.0,339.0,7484.482500,1.028444,7975.439335
19409,2.0,141.0,234.0,9060.610000,0.777964,7861.118568
16863,3.0,170.0,247.0,7383.973333,0.965721,7568.390197


In [43]:
clv['Segment'] =  pd.qcut(clv['six_Months_CLV'],4,labels = ['Hibernating','Need Attention', 'LoyalCustomers', 'Champions'])

In [44]:
clv.head()

,frequency,recency,T,monetary_value,expected_purc_6_months,six_Months_CLV,Segment
Customer ID,,,,,,,
1038,2.0,77.0,154.0,2841.790000,0.881826,2796.311574,LoyalCustomers
1060,2.0,123.0,438.0,1532.280000,0.150586,257.440594,Hibernating
1079,2.0,327.0,412.0,2769.165000,0.740819,2287.939442,LoyalCustomers
1101,3.0,288.0,427.0,298.313333,0.654165,207.715313,Hibernating
1134,2.0,192.0,374.0,2319.800000,0.444792,1150.970062,Need Attention


In [45]:
clv.groupby('Segment').mean()

,frequency,recency,T,monetary_value,expected_purc_6_months,six_Months_CLV
Segment,,,,,,
Hibernating,2.154506,153.223176,358.665236,1546.928222,0.400506,541.572227
Need Attention,2.214592,162.811159,316.768240,2715.821391,0.551422,1455.183830
LoyalCustomers,2.262931,177.616379,310.284483,3646.833473,0.630112,2384.900710
Champions,2.300429,176.167382,286.351931,5224.043115,0.724240,4066.477272


Interpret the segmentation results.
Explain the characteristics of each customer segment.
Suggest business strategies based on the RFM clusters.


Hibernating Segment: lowest frequency, and lowest recency as well as very low monetary value and low CLV
These are customers that have not purchased in a long time and don't spend much, probably not repeat customers, one time customers.

Need Attention Segment: higher frequency than Hibernating, better recency but are not buying much. Could be repeat customers, but might need incentive like coupons to get them back in.

Loyal Customer Segment: frequency in the middle with consistent returns and they spend consistently. Regular customers, trusted this store for their needs and sees you as their go to source.

Champions: Strongest in monetary value, good recency and solid frequency. highest CLV in all the groups. Regular spenders. Deeply engaged, statisfied, potential brand ambasadors. could be bulk or wholesale buyers receiving a discount.